In [1]:
import csv
import sys
import statistics
from numpy import percentile

In [2]:
csv_file = open('../dataset/FilmesBrasileirosLancados.csv', 'r')
csv_reader = csv.reader(csv_file, delimiter=';')
next(csv_reader) # skip header

['"Ano de Lançamento"',
 '"Certificado de Produto Brasileiro (CPB)"',
 '"Título"',
 '"Direção"',
 '"Gênero"',
 '"Empresa Produtora Brasileira Majoritária"',
 '"UF"',
 '"Empresa Produtora Minoritária Brasileira"',
 '"UF"',
 '"Distribuidora"',
 '"Máximo de Salas"',
 '"Renda (R$)"',
 '"Público "']

## Filtrando campos necessários

In [3]:
def edit_text(rstring):
    rstring = rstring.lower()
    rstring = rstring.replace('"', '').replace("ç","c")
    rstring = rstring.replace("á", "a").replace("é", "e").replace("í","i").replace("ó","o").replace("ú","u")
    rstring = rstring.replace("ã", "a").replace("õ","o")
    rstring = rstring.replace("â", "a").replace("ê", "e").replace("î","i").replace("ô","o").replace("û","u")
    return rstring

In [4]:
data = []
for row in csv_reader:
    if row[0] != '':
        if row[11] != '"ND"' and row[12] != '"ND"': # Removing movies that don't have all information
            movie = {"Diretor": row[3].replace('"', ''),"Genero": edit_text(row[4]),"Distribuidora": row[9].replace('"', ''),"Renda": float(row[11].replace('"', '').replace(',', '.')),"Publico": int(row[12].replace('"', ''))}
            data.append(movie)

In [5]:
print(len(data))

1532


## Conversão Atributos Nominais

Diretores

In [6]:
director_lst = []
director_name = []
for r in data:
    if r['Diretor'] not in director_name:
        director_name.append(r['Diretor'])
        director_lst.append({"Diretor": r['Diretor'],"Renda": [r['Renda']],"Publico": [r['Publico']]})
    else:
        director_lst[director_name.index(r['Diretor'])]['Renda'].append(r['Renda'])
        director_lst[director_name.index(r['Diretor'])]['Publico'].append(r['Publico'])

In [7]:
for r in data:
    if r['Diretor'] in director_name:
        r['Diretor'] = statistics.median(director_lst[director_name.index(r['Diretor'])]['Renda'])
    else:
        r['Diretor'] = 'Outros'

Genêro

In [8]:
genre_lst = []
for r in data:
    if r['Genero'] not in genre_lst:
        genre_lst.append(r['Genero'])

In [9]:
len(genre_lst)

4

In [10]:
for r in data:
    r['Genero'] = genre_lst.index(r['Genero']) + 1

Distribuidora

In [11]:
csv_file2 = open('../dataset/DistribuidorasFilmesBrasil.csv', 'r')
csv_reader2 = csv.reader(csv_file2, delimiter=';')
next(csv_reader2) # skip header

['"Ano de exibição"',
 '"Distribuidora"',
 '"Número de Registro Ancine"',
 '"Origem da empresa distribuidora"',
 '"Público"',
 '"Público Filmes Brasileiros"',
 '"Renda (R$)"',
 '"Renda Filmes Brasileiros (R$)"',
 '"Títulos Exibidos"',
 '"Títulos Brasileiros  Exibidos"',
 '"Títulos Lançados"',
 '"Títulos Brasileiros Lançados"']

In [12]:
data2 = []
for row in csv_reader2:
    movie = {"Distribuidora": row[1].replace('"', ''),"Renda": float(row[6].replace('"', '').replace(',', '.')),"Lancamentos": int(row[8].replace('"', ''))}
    data2.append(movie)

In [13]:
distribuidora_lst = []
distribuidora_name = []
for r in data2:
    if r['Distribuidora'] not in distribuidora_name:
        distribuidora_name.append(r['Distribuidora'])
        distribuidora_lst.append(r)
    else:
        distribuidora_lst[distribuidora_name.index(r['Distribuidora'])]['Renda'] += r['Renda']
        distribuidora_lst[distribuidora_name.index(r['Distribuidora'])]['Lancamentos'] += r['Lancamentos']

In [14]:
for d in director_name:
    director_lst[director_name.index(d)]['Renda'] = statistics.median(director_lst[director_name.index(d)]['Renda'])

In [15]:
for r in data:
    if r['Distribuidora'] in distribuidora_name:
        r['Distribuidora'] = distribuidora_lst[distribuidora_name.index(r['Distribuidora'])]['Renda']
    else:
        r['Distribuidora'] = 0

## Exportando resultados

In [16]:
csv_out = []
for r in data:
    row = []
    row.append(r['Diretor'])
    row.append(r['Genero'])
    row.append(r['Distribuidora'])
    row.append(r['Publico'])
    csv_out.append(row)

In [17]:
with open('../dataset/RegressionAnalysisPreProcessing.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(csv_out)